In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable

In [2]:
labels = pd.read_csv("../../data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


#### Perform mean, median, max, min and sum pooling on audio feature data

In [4]:
def get_model_data(audio_features):
    model_data = pd.DataFrame(columns=['audio_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
        audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'audio_feature': audio_features[audio_key],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [5]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data

In [6]:
class RNNTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'padded_audio_feature']
            a=np.empty((18,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'padded_audio_feature']
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)
    
class RNNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
class RNNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
def evaluateRNN(rnn, review, size):
    output = rnn(review)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(rnn, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateRNN(rnn, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals

In [7]:
with open('../../audio_features/feat_dict_librosa_lld.pickle', 'rb') as f:
    librosa_audio_features = pickle.load(f, encoding='latin1')
    
model_data = get_model_data(librosa_audio_features)
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,audio_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[-386.6164855957031, -649.6673512776692, -633...",0.0,NONE,SHELDON,25
1,"[[-255.5221405029297, -484.69307309105284, -52...",0.0,NONE,PENNY,15
2,"[[-569.0548095703125, -381.4147456242488, -221...",0.0,NONE,RAJ,21
3,"[[-237.61074829101562, -211.002773845897, -382...",1.0,PRO,HOWARD,7
4,"[[-530.5701293945312, -374.83951552370763, -42...",0.0,NONE,SHELDON,25


In [8]:
train_data, test_data = get_train_test_split(model_data, ['audio_feature', 'speaker_encode'], 'sarcasm', 'sarcasm_type')
rnn_train = train_data.copy()
rnn_test = test_data.copy()
rnn_train.reset_index(drop=True, inplace = True)
rnn_test.reset_index(drop=True, inplace = True)

Train:  (961, 2) (961,) Test:  ((241, 2), (241,))
<class 'pandas.core.frame.DataFrame'>


In [9]:
desired_length = 18

rnn_train['padded_audio_feature'] = rnn_train.loc[:, 'audio_feature']
for index, row in rnn_train.iterrows():
    data_array = row['padded_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length:
            arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    rnn_train.at[index, "padded_audio_feature"] = np.transpose(np.array(new_array))

rnn_test['padded_audio_feature'] = rnn_test.loc[:, 'audio_feature']
for index, row in rnn_test.iterrows():
    data_array = row['padded_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length:
            arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    rnn_test.at[index, "padded_audio_feature"] = np.transpose(np.array(new_array))

rnn_train["sarcasm"] = rnn_train["sarcasm"].astype('int').to_numpy()
rnn_test["sarcasm"] = rnn_test["sarcasm"].astype('int').to_numpy()

In [10]:
EMBEDDING_DIM_sid = 690
EMBEDDING_DIM_sd = 691
HIDDEN_DIM = 20
OUTPUT_DIM = 2
layers = 2
criterion = nn.NLLLoss()

### Speaker InDependent

In [11]:
rnn_train_tensor = RNNTensorDataset(rnn_train[['padded_audio_feature', 'sarcasm']], False)
rnn_test_tensor = RNNTensorDataset(rnn_test[['padded_audio_feature', 'sarcasm']], False)

num_of_workers = 0
batch_size = 44
valid_size = 0.1

train_indices = list(range(len(rnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(rnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    rnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    rnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(
    rnn_test_tensor, batch_size=rnn_test_tensor.__len__())

In [12]:
rnn = RNNetSID(EMBEDDING_DIM_sid, HIDDEN_DIM, OUTPUT_DIM, layers)
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)

n_epochs = 501
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    rnn.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    rnn.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        output = rnn(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(rnn, test_loader_epoch, rnn_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits=4))

RNNetSID(
  (rnn): RNN(690, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)
Epoch: 0
              precision    recall  f1-score   support

           0     0.4958    0.9917    0.6611       120
           1     0.0000    0.0000    0.0000       121

    accuracy                         0.4938       241
   macro avg     0.2479    0.4958    0.3306       241
weighted avg     0.2469    0.4938    0.3292       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.6765    0.5750    0.6216       120
           1     0.6331    0.7273    0.6769       121

    accuracy                         0.6515       241
   macro avg     0.6548    0.6511    0.6493       241
weighted avg     0.6547    0.6515    0.6494       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.6281    0.6333    0.6307       120
           1     0.6333    0.6281    0.6307       121

    accuracy                 

Epoch: 480
              precision    recall  f1-score   support

           0     0.5897    0.5750    0.5823       120
           1     0.5887    0.6033    0.5959       121

    accuracy                         0.5892       241
   macro avg     0.5892    0.5892    0.5891       241
weighted avg     0.5892    0.5892    0.5891       241

Epoch: 500
              precision    recall  f1-score   support

           0     0.6455    0.5917    0.6174       120
           1     0.6260    0.6777    0.6508       121

    accuracy                         0.6349       241
   macro avg     0.6357    0.6347    0.6341       241
weighted avg     0.6357    0.6349    0.6342       241



### Speaker Dependent

In [15]:
rnn_train_tensor = RNNTensorDataset(rnn_train[['padded_audio_feature', 'speaker_encode','sarcasm']], True)
rnn_test_tensor = RNNTensorDataset(rnn_test[['padded_audio_feature', 'speaker_encode', 'sarcasm']], True)

num_of_workers = 0
batch_size = 44
valid_size = 0.1

train_indices = list(range(len(rnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(rnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    rnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    rnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(
    rnn_test_tensor, batch_size=rnn_test_tensor.__len__())

In [16]:
rnn = RNNetSD(EMBEDDING_DIM_sd, HIDDEN_DIM, OUTPUT_DIM, layers)
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)

n_epochs = 501
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    rnn.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    rnn.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        output = rnn(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%10 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(rnn, test_loader_epoch, rnn_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits=4))

RNNetSD(
  (rnn): RNN(691, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)
Epoch: 0
              precision    recall  f1-score   support

           0     0.4979    1.0000    0.6648       120
           1     0.0000    0.0000    0.0000       121

    accuracy                         0.4979       241
   macro avg     0.2490    0.5000    0.3324       241
weighted avg     0.2479    0.4979    0.3310       241

Epoch: 10
              precision    recall  f1-score   support

           0     0.6667    0.6167    0.6407       120
           1     0.6462    0.6942    0.6693       121

    accuracy                         0.6556       241
   macro avg     0.6564    0.6554    0.6550       241
weighted avg     0.6564    0.6556    0.6551       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.6723    0.6667    0.6695       120
           1     0.6721    0.6777    0.6749       121

    accuracy                  

Epoch: 240
              precision    recall  f1-score   support

           0     0.5660    0.7500    0.6452       120
           1     0.6341    0.4298    0.5123       121

    accuracy                         0.5892       241
   macro avg     0.6001    0.5899    0.5787       241
weighted avg     0.6002    0.5892    0.5785       241

Epoch: 250
              precision    recall  f1-score   support

           0     0.5856    0.5417    0.5628       120
           1     0.5769    0.6198    0.5976       121

    accuracy                         0.5809       241
   macro avg     0.5813    0.5808    0.5802       241
weighted avg     0.5812    0.5809    0.5803       241

Epoch: 260
              precision    recall  f1-score   support

           0     0.5816    0.6833    0.6284       120
           1     0.6200    0.5124    0.5611       121

    accuracy                         0.5975       241
   macro avg     0.6008    0.5979    0.5947       241
weighted avg     0.6009    0.5975    0.59

Epoch: 490
              precision    recall  f1-score   support

           0     0.5748    0.6083    0.5911       120
           1     0.5877    0.5537    0.5702       121

    accuracy                         0.5809       241
   macro avg     0.5813    0.5810    0.5807       241
weighted avg     0.5813    0.5809    0.5806       241

Epoch: 500
              precision    recall  f1-score   support

           0     0.5748    0.6083    0.5911       120
           1     0.5877    0.5537    0.5702       121

    accuracy                         0.5809       241
   macro avg     0.5813    0.5810    0.5807       241
weighted avg     0.5813    0.5809    0.5806       241

